In [8]:
import numpy as np
import cvxpy as cp
import random

In [13]:
def generate_support(nmaps, nqubits, nmaps_per_qubit, qubit_occupancies):
    mat = cp.Variable(shape=(nmaps,nqubits), boolean = True)
    constraints = []
    col_sums = cp.sum(mat, axis=0, keepdims=True)
    constraints.append(col_sums >= 1)
    row_sums = cp.sum(mat, axis=1)
    constraints.append(row_sums == qubit_occupancies)
    objective = cp.Minimize(cp.norm(col_sums - nmaps_per_qubit, "fro"))
    problem = cp.Problem(objective, constraints)
    problem.solve(parallel=True, verbose=False)
    print(problem.status)
    if ("optimal" in problem.status):
        support = [tuple(np.nonzero(np.round(row).astype(np.int))[0]) for row in mat.value]
        print("Optimization value = {}".format(problem.value))
        print("Occupancy matrix = {}".format(mat.value))
        print("Number of maps per qubit : {}".format(np.sum(mat.value,axis=0)))
        print("Support : {}".format(support))
    else:
        print("Qubit allocation to maps infeasible.")

In [16]:
nmaps = 2
nqubits = 7
nmaps_per_qubit = max(0.1*nmaps,1)
qubit_occupancies = np.array([ random.randint(1, nqubits//2) for __ in range(nmaps)])
print("Qubit occupancies : {}".format(qubit_occupancies))
generate_support(nmaps, nqubits, nmaps_per_qubit, qubit_occupancies)

Qubit occupancies : [3 1]
infeasible_inaccurate
Qubit allocation to maps infeasible.


In [26]:
np.round(1E-5)

0.0

In [4]:
import sys
print (sys.version)

3.7.1 (v3.7.1:260ec2c36a, Oct 20 2018, 03:13:28) 
[Clang 6.0 (clang-600.0.57)]


In [24]:
[np.random.poisson(lam=1.5) for __ in range(10)]

[5, 3, 2, 4, 3, 2, 1, 1, 2, 2]